In [ ]:
%env CUDA_VISIBLE_DEVICES = 1

In [2]:
%load_ext autoreload
%autoreload 2
import sys; 
sys.path.extend(['/home/meet/FlowMatchingTests/conditional-flow-matching/'])

import matplotlib.pyplot as plt
import numpy as np
import torch
from torchcfm.conditional_flow_matching import *
from physics_flow_matching.unet import UNetModel
from torchdiffeq import odeint
from einops import rearrange

In [ ]:
label = np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_u_y70_41-220_out.npy")
# test_label = np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_u_y70_221-259_out.npy")
m, std = label.mean(axis=(0,2,3), keepdims=True), label.std(axis=(0,2,3), keepdims=True)
# Y = (test_label - m)/std
# test_label = rearrange(test_label, "b l c w -> (b c) l w")
# test_label = (test_label[:, 160:])[:, None]
# Y = (label - label.mean())/label.std()

In [4]:
l_d = (label - m)/std

In [5]:
data = np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_wall_dudy_41-220_out.npy")
m_, std_ = data.mean(), data.std()

test_data = np.load("/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_wall_dudy_221-259_out.npy")
# test_data = rearrange(test_data, "b l c w -> (b c) l w")
# test_data = (test_data[:, 160:])[:, None]
X = (test_data - m_)/std_

In [6]:
X_d = (data - m_)/std_

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
exp=4
iteration = 5
ot_cfm_model = UNetModel(dim=[1, 160, 200],
                        out_channels=1,
                        num_channels=128,
                        channel_mult = None,
                        num_res_blocks=2,
                        num_head_channels=64,
                        attention_resolutions="40, 20",
                        dropout=0.2,
                        use_new_attention_order=True,
                        use_scale_shift_norm=True)
state = torch.load(f"/home/meet/FlowMatchingTests/conditional-flow-matching/physics_flow_matching/ws_vf/exps/exp_{exp}/saved_state/checkpoint_{iteration}.pth")
ot_cfm_model.load_state_dict(state["model_state_dict"])
ot_cfm_model.to(device)
ot_cfm_model.eval();

In [9]:
batch_size=10

In [ ]:
wall_norm_dict = {0:0}
key = 1

In [ ]:
samples_list = []
for batch in range(X.shape[0]//batch_size):
    with torch.no_grad():
        out = ot_cfm_model(torch.tensor(wall_norm_dict[key]*np.ones(batch_size).astype(np.float32), device=device)[:, None], torch.tensor(X[batch*batch_size:(batch+1)*batch_size][:, None], device=device)).detach().cpu().numpy()
    out *= std[:, key]
    out += m[:, key]
    samples_list.append(out)

In [26]:
samples = np.concatenate(samples_list)

In [27]:
np.save(f"/home/meet/FlowMatchingTests/conditional-flow-matching/physics_flow_matching/ws_vf/exps/exp_{exp}/y_{wall_norm_dict[key]}_samples_{iteration}epch", samples[:, 0])

In [ ]:
# for i in range(5):
#     fig, axes = plt.subplots(nrows=1, ncols=4, dpi=200)
#     ax = axes[0].imshow(X_d[200+i,0])
#     plt.colorbar(ax,ax=axes[0])
#     ax = axes[1].imshow(samples[i,0])
#     plt.colorbar(ax,ax=axes[1])
#     ax= axes[2].imshow(label[200+i, 1])
#     plt.colorbar(ax,ax=axes[2])
#     ax = axes[3].imshow(np.abs(samples[i,0] - label[200+i, 1]))
#     plt.colorbar(ax,ax=axes[3])

In [ ]:
# plt.imshow(X_d[0,0] - X_d[10,0])
# plt.colorbar()